In [1]:
# IMPORT LIBRARIES
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, random, button, label, box, cross, mag, arrow, sleep, local_light, cone, pyramid, compound

<IPython.core.display.Javascript object>

In [2]:
# the create bird function
def create_bird():
    rnd_h = random()
    R=0.3
    r=0.19
    h_p = 0.3
    nose = cone(pos=vector((R+r)*np.sqrt(2)/2+r, rnd_h+(R+r)*np.sqrt(2)/2,0),axis=vector(0.3,0,0),radius=0.1,color=color.yellow)
    body = sphere(pos=vector(0,rnd_h,0),color=color.red, radius = R)
    head = sphere(pos=vector((R+r)*np.sqrt(2)/2, rnd_h+(R+r)*np.sqrt(2)/2,0), radius=r,color=0.01*vector(255,69,0))
    tail = pyramid(pos=vector(-r-h_p,rnd_h,0),size=vector(0.25,0.25,h_p),color=color.black)
    bird = compound([nose, body, head, tail])
    bird.make_trail = True
    bird.trail_type = "points"
    bird.interval  = 10
    bird.trail_color = 0.01*vector(66.3, 66.3, 66.3)

In [ ]:
# FROM THE PDF
scene = canvas(width=640, height=480, center=vector(8,5,0),range=8,background = 0.01*vector(39.6, 84.7, 100))
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)



# SET INITIAL VALUES
g = 9.81
contact_t = 0.01
step_t = 0.0001
m_bird = 0.1
m_box = 100
t = 0
posX = 0
posY = 0
no_topple = True
attempts_no = 0
rate(5000)

# CREATE TARGET, BIRD, LABEL1 AND MOMENTUM VECTOR

# target
rnd_X = 5 + random()*10
target_box = box(pos=vector(rnd_X,1,0), length=0.5, height=2, width=0.5,color=0.01*vector(62.4, 41.2, 20.4))

# bird
create_bird()

# momentum vector
momentum_arrow = arrow(pos=vector(posX,posY,0), axis=vector(0,0,0))

# create label 1
label1 = label(pos=vector(10,10,0), text='', visible = False)

# wait to load the objects
sleep(2)


# MAIN LOOP OF THE GAME, GOES ON UNTIL THE TARGET IS TOPPLED
while no_topple:
    
    # increase the number of attempts
    attempts_no += 1
    
    # RESET THE VISUALS
    # disappear the label
    label1.visible = False
    # reset bird's position
    bird.pos=vector(0,rnd_h,0)    
    # "delete" the old momentum arrow
    momentum_arrow.visible = False   
    # wait for the visuals to reset
    sleep(1)
    
    # input initial conditions (angle and speed)
    dtheta = float(input("Input the initial angle in degrees: "))
    theta = np.radians(dtheta)
    v0 = float(input("Input the initial speed in metres/second: "))
    
    # calculate Vx and Vy from the user input v0
    vX = v0*np.cos(theta)
    vY = v0*np.sin(theta)
    
    # reset posY, posX, t and hit
    posY = rnd_h
    posX = 0
    t=0
    hit =  False
    
    # calculate the new momentum vector and display it
    momentum_arrow.axis = m_bird*vector(vX,vY-g*t,0)
    momentum_arrow.visible = True
    
    # START ANIMATION
    while posY >= 0 and posX<=16: 
        rate(5000)
        t+=0.0001
        posY = rnd_h + vY*t - g/2*t**2                         #into function
        posX = vX*t
        bird.pos = vector(posX,posY,0)
        momentum_arrow.pos = vector(posX,posY,0)
        momentum_arrow.axis = m_bird*vector(vX,vY-g*t,0)
        #sleep(0.0000001)
               
        # Calculating if its a hit or not
        if ((rnd_X-0.5)<=posX<=(rnd_X+0.5)) and (0<=posY<=2):
            
            hit = True
            label1 = label(pos=vector(10,10,0), text='It is a hit!')
            
            # Calculating if it will fall
            rest_T = m_box*g*0.5*0.5
            F = m_bird*vector(vX,vY-g*t,0)/contact_t # 10*vector(...)      ###put this into a funciton
            appl_T = cross(F,vector(posX-rnd_X+0.25,posY,0))
            mag_appl_T = mag(appl_T)
            
            # print statements to inform the user
            print("The ball hit the target!")
            print("The height of the impact is {0}m".format(posY))
            print("The bird's momentum at the point of impact: {0}kgm/s".format(mag(m_bird*vector(vX,vY-g*t,0))))   ###func
            print("The applied torque is {0}Nm".format(mag_appl_T))
            print("The restoring torque is {0}Nm".format(rest_T))
            
            #arrow2 = arrow(pos=vector(rnd_X,0,0), axis=vector(posX-rnd_X+0.25,posY,0))
            if mag_appl_T>rest_T:
                print("IT WILL TOPPLE!!")
                print("----------------------------")
                no_topple = False
                break
            else:
                print("IT WONT TOPPLE :( ")
                print("----------------------------")
                break
    
    # Output these if the bird did not hit the target
    if hit==False:
        print(" No hit, try again!")
        print("The ball landed at x= {0}".format(posX))
        print("----------------------------")
            

#target_box.visible = False
momentum_arrow.visible = False
label1.text = "You have toppled the box"

# the box falling animation
rate(1500)
hit_body = compound([bird, target_box])
angle_ = 0.01
while angle_ < 3.141592/2:     ####into function
    rate(1500)
    angle_+=0.001
    hit_body.rotate(angle=-0.001, origin = vector(rnd_X+0.25,0,0.25), axis = vector(0,0,0.5))
    



<IPython.core.display.Javascript object>